In [5]:
from os import environ
import dotenv

dotenv.load_dotenv("../.env")
GROQ_API=environ.get('GROQ_API')

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("vendas.csv")

In [8]:
df.head()

,ID_compra,filial,cidade,tipo_cliente,genero,tipo_produto,preco_unitario,quantidade,imposto_5%,total,data,hora,forma_pagamento,avaliacao
0,750-67-8428,A,Santo André,Membro,Feminino,Saúde e Beleza,74.69,7,26.1415,548.9715,2024-01-05,13:08:00,Carteira Digital,9.1
1,226-31-3081,C,São Caetano,Normal,Feminino,Eletrônicos,15.28,5,3.8200,80.2200,2024-03-08,10:29:00,Dinheiro,9.6
2,631-41-3108,A,Santo André,Normal,Masculino,Casa,46.33,7,16.2155,340.5255,2024-03-03,13:23:00,Cartão de Crédito,7.4
3,123-19-1176,A,Santo André,Membro,Masculino,Saúde e Beleza,58.22,8,23.2880,489.0480,2024-01-27,20:33:00,Carteira Digital,8.4
4,373-73-7910,A,Santo André,Normal,Masculino,Esportes e Viagem,86.31,7,30.2085,634.3785,2024-02-08,10:37:00,Carteira Digital,5.3


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID_compra        1000 non-null   object 
 1   filial           1000 non-null   object 
 2   cidade           1000 non-null   object 
 3   tipo_cliente     1000 non-null   object 
 4   genero           1000 non-null   object 
 5   tipo_produto     1000 non-null   object 
 6   preco_unitario   1000 non-null   float64
 7   quantidade       1000 non-null   int64  
 8   imposto_5%       1000 non-null   float64
 9   total            1000 non-null   float64
 10  data             1000 non-null   object 
 11  hora             1000 non-null   object 
 12  forma_pagamento  1000 non-null   object 
 13  avaliacao        1000 non-null   float64
dtypes: float64(4), int64(1), object(9)
memory usage: 109.5+ KB


In [10]:
df.duplicated().sum()

np.int64(0)

In [11]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

Settings.llm = Groq(model='llama-3.3-70b-versatile', api_key=GROQ_API)

In [12]:
from llama_index.core import Settings

In [13]:
from llama_index.experimental.query_engine import PandasQueryEngine

In [14]:
query_engine = PandasQueryEngine(df=df, verbose=True)

In [15]:
response = query_engine.query("qual a forma de pagamento mais utilizada pelos clientes?")

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df['forma_pagamento'].mode()[0]
```
> Pandas Output: Carteira Digital


In [11]:
df['forma_pagamento'].value_counts()

forma_pagamento
Carteira Digital     345
Dinheiro             344
Cartão de Crédito    311
Name: count, dtype: int64

In [12]:
response = query_engine.query('Qual é o tipo de produto com maior quantidade por filial?')

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.groupby('filial')['tipo_produto'].value_counts().groupby(level=0).apply(lambda x: x.idxmax())
```
> Pandas Output: filial
A                   (A, Casa)
B      (B, Esportes e Viagem)
C    (C, Alimentos e Bebidas)
Name: count, dtype: object


In [13]:
query_engine.query('Em quais cidades temos filiais?').response

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df['cidade'].unique()
```
> Pandas Output: ['Santo André' 'São Caetano' 'São Bernardo do Campo']


"['Santo André' 'São Caetano' 'São Bernardo do Campo']"

In [14]:
query_engine.query('Qual é o preço unitário médio de cada tipo de produto?').response

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.groupby('tipo_produto')['preco_unitario'].mean()
```
> Pandas Output: tipo_produto
Alimentos e Bebidas    56.008851
Casa                   55.316937
Eletrônicos            53.551588
Esportes e Viagem      56.993253
Moda                   57.153652
Saúde e Beleza         54.854474
Name: preco_unitario, dtype: float64


'tipo_produto\nAlimentos e Bebidas    56.008851\nCasa                   55.316937\nEletrônicos            53.551588\nEsportes e Viagem      56.993253\nModa                   57.153652\nSaúde e Beleza         54.854474\nName: preco_unitario, dtype: float64'

In [15]:
query_engine.query('Qual é a média de valor de compra por tipo de cliente?').response

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.groupby('tipo_cliente')['total'].mean()
```
> Pandas Output: tipo_cliente
Membro    327.791305
Normal    318.122856
Name: total, dtype: float64


'tipo_cliente\nMembro    327.791305\nNormal    318.122856\nName: total, dtype: float64'

In [16]:
query_engine.query('Qual é a filial com maior faturamento?ss').response

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.groupby('filial')['total'].sum().idxmax()
```
> Pandas Output: C


'C'

In [17]:
query_engine = PandasQueryEngine(df=df, verbose=True, synthesize_response=True)

In [18]:
response = query_engine.query('Qual é a avaliação média de cada filial?')
print(str(response))

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.groupby('filial')['avaliacao'].mean()
```
> Pandas Output: filial
A    7.027059
B    6.818072
C    7.072866
Name: avaliacao, dtype: float64
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
A avaliação média de cada filial é a seguinte: 
- Filial A: 7.03
- Filial B: 6.82
- Filial C: 7.07

Esses valores indicam a média das avaliações recebidas por cada filial, fornecendo uma visão geral do desempenho de cada uma.


In [19]:
import textwrap

def formatar_texto(response):
    texto = response.response
    texto_formatado = textwrap.fill(texto, width=100)
    print(texto_formatado)

In [20]:
formatar_texto(response)

A avaliação média de cada filial é a seguinte:  - Filial A: 7.03 - Filial B: 6.82 - Filial C: 7.07
Esses valores indicam a média das avaliações recebidas por cada filial, fornecendo uma visão geral
do desempenho de cada uma.


In [21]:
response = query_engine.query('Qual é a avaliação média de cada filial? Ordene da maior para a menor')
print(response.response)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.groupby('filial')['avaliacao'].mean().sort_values(ascending=False)
```
> Pandas Output: filial
C    7.072866
A    7.027059
B    6.818072
Name: avaliacao, dtype: float64
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
A avaliação média de cada filial, em ordem da maior para a menor, é a seguinte:

- Filial C: 7.07
- Filial A: 7.03
- Filial B: 6.82

Esses valores indicam que a filial C tem a avaliação média mais alta, seguida pela filial A e, por fim, pela filial B.


In [22]:
response = query_engine.query('Você pode exibir a distribuição das avaliações?')
formatar_texto(response)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df['avaliacao'].value_counts()
```
> Pandas Output: avaliacao
6.0     26
6.6     24
9.5     22
4.2     22
5.1     21
        ..
5.3     11
8.3     11
4.0     11
4.6      8
10.0     5
Name: count, Length: 61, dtype: int64
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Sim, posso exibir a distribuição das avaliações. Com base nos resultados, podemos observar que as
avaliações mais comuns são:  - 6.0, que aparece 26 vezes - 6.6, que aparece 24 vezes - 9.5 e 4.2,
que ambos aparecem 22 vezes - 5.1, que aparece 21 vezes  As avaliações menos comuns incluem: - 5.3,
8.3 e 4.0, que aparecem 11 vezes cada - 4.6, que aparece 8 vezes - 10.0, que aparece apenas 5 

In [23]:
response = query_engine.query('Você pode plotar a distribuição das avaliações?')
formatar_texto(response)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df['avaliacao'].plot.hist(bins=10, figsize=(10, 6), title='Distribuição das Avaliações')
```
> Pandas Output: There was an error running the output as Python code. Error message: matplotlib is required for plotting when the default backend "matplotlib" is selected.


Traceback (most recent call last):
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 63, in default_output_processor
    output_str = str(safe_eval(module_end_str, global_vars, local_vars))
                     ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\llama_index\experimental\exec_utils.py", line 159, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
  File "<string>", line 1, in <module>
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\plotting\_core.py", line 1409, in hist
    return self(kind="hist", by=by, bins=bins, **kwargs)
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\plotting\_core.py", line 947, in __call__
    plot_backend = _get_plot_backend(kwargs.pop("backend", None))
  

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Sim, é possível plotar a distribuição das avaliações. Para fazer isso, você pode utilizar a
biblioteca Pandas em conjunto com a biblioteca Matplotlib para criar um histograma que mostre a
distribuição das avaliações.  Aqui está um exemplo de como você pode fazer isso:  ```python import
pandas as pd import matplotlib.pyplot as plt  # Supondo que você tenha um DataFrame chamado 'df' com
uma coluna 'avaliacao' df['avaliacao'].plot.hist(bins=10, figsize=(10, 6), title='Distribuição das
Avaliações') plt.show() ```  Nesse código:  - `df['avaliacao']` seleciona a coluna 'avaliacao' do
DataFrame. - `plot.hist()` cria um histograma da distribuição das avaliações. - `bins=10` define o
número de intervalos (ou "caixas") no histograma. - `figsize=(10, 6)` define o tamanho da figura em
polegadas. - `title='Distribuição d

In [24]:
response = query_engine.query('''
Você pode plotar o valor total por tipo de produto, ordenando do menor para o maior e mantendo as barras na horizontal?
''')
formatar_texto(response)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.groupby('tipo_produto')['total'].sum().sort_values(ascending=True).plot(kind='barh')
```
> Pandas Output: There was an error running the output as Python code. Error message: matplotlib is required for plotting when the default backend "matplotlib" is selected.


Traceback (most recent call last):
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 63, in default_output_processor
    output_str = str(safe_eval(module_end_str, global_vars, local_vars))
                     ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\llama_index\experimental\exec_utils.py", line 159, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
  File "<string>", line 1, in <module>
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\plotting\_core.py", line 947, in __call__
    plot_backend = _get_plot_backend(kwargs.pop("backend", None))
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\plotting\_core.py", line 1944, in _get_plot_backend
    module = _load_backend(backend_str)
  File

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Sim, é possível plotar o valor total por tipo de produto, ordenando do menor para o maior e mantendo
as barras na horizontal. Para fazer isso, você pode usar a biblioteca pandas em conjunto com a
biblioteca matplotlib para criar o gráfico.  Aqui está um exemplo de como você pode fazer isso:
```python import pandas as pd import matplotlib.pyplot as plt  # Supondo que você tenha um DataFrame
chamado 'df' com as colunas 'tipo_produto' e 'total' df = pd.DataFrame({     'tipo_produto': ['A',
'B', 'A', 'C', 'B', 'C'],     'total': [10, 20, 15, 30, 25, 35] })  # Agrupe os dados por
'tipo_produto' e some os valores de 'total' valor_total_por_tipo =
df.groupby('tipo_produto')['total'].sum()  # Ordene os valores do menor para o maior
valor_total_por_tipo = valor_total_por_tipo.sort_values(ascending=True)  # Crie o grá

In [25]:
response = query_engine.query('''Plote o valor total por tipo de produto,
ordenando do menor para o maior e mantendo as barras na horizontal''')
formatar_texto(response)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.groupby('tipo_produto')['total'].sum().sort_values(ascending=True).plot(kind='barh')
```
> Pandas Output: There was an error running the output as Python code. Error message: matplotlib is required for plotting when the default backend "matplotlib" is selected.


Traceback (most recent call last):
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\llama_index\experimental\query_engine\pandas\output_parser.py", line 63, in default_output_processor
    output_str = str(safe_eval(module_end_str, global_vars, local_vars))
                     ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\llama_index\experimental\exec_utils.py", line 159, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
  File "<string>", line 1, in <module>
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\plotting\_core.py", line 947, in __call__
    plot_backend = _get_plot_backend(kwargs.pop("backend", None))
  File "c:\Users\leoka\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\plotting\_core.py", line 1944, in _get_plot_backend
    module = _load_backend(backend_str)
  File

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Para resolver o problema e plotar o valor total por tipo de produto, ordenando do menor para o maior
e mantendo as barras na horizontal, você precisará ter o matplotlib instalado em seu ambiente
Python. Aqui está uma resposta detalhada sobre como fazer isso:  1. **Instalar o Matplotlib**: Se
você ainda não tem o matplotlib instalado, pode fazê-lo usando pip, que é o gerenciador de pacotes
do Python. Abra seu terminal ou prompt de comando e execute o seguinte comando:    ```    pip
install matplotlib    ```    Se você estiver usando um ambiente virtual, certifique-se de que ele
esteja ativado antes de instalar.  2. **Importar Bibliotecas Necessárias**: No início do seu script
Python, você precisará importar as bibliotecas necessárias. Isso inclui `pandas` para manipulação de
dados e `matplotlib.pyplot` para p

In [26]:
response = query_engine.query('Qual é o método de pagamento mais utilizado em cada filial?')
print(response.response)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.groupby('filial')['forma_pagamento'].apply(lambda x: x.mode().iloc[0])
```
> Pandas Output: filial
A    Carteira Digital
B    Carteira Digital
C            Dinheiro
Name: forma_pagamento, dtype: object
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
O método de pagamento mais utilizado em cada filial é o seguinte:

- Na filial A, o método de pagamento mais utilizado é a Carteira Digital.
- Na filial B, o método de pagamento mais utilizado também é a Carteira Digital.
- Na filial C, o método de pagamento mais utilizado é o Dinheiro.

Esses resultados indicam que as filiais A e B têm uma preferência por pagamentos digitais, enquanto a filial C ainda 

In [27]:
response = query_engine.query('Qual é o método de pagamento mais utilizado em cada filial? Responda em português BR')
print(response.response)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.groupby('filial')['forma_pagamento'].apply(lambda x: x.mode().iloc[0])
```
> Pandas Output: filial
A    Carteira Digital
B    Carteira Digital
C            Dinheiro
Name: forma_pagamento, dtype: object
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
O método de pagamento mais utilizado em cada filial é o seguinte:

- Na filial A, o método mais utilizado é a Carteira Digital.
- Na filial B, o método mais utilizado também é a Carteira Digital.
- Na filial C, o método mais utilizado é o Dinheiro.

Esses resultados indicam que as filiais A e B têm uma preferência por métodos de pagamento digitais, enquanto a filial C ainda tem uma preferência por pagam

In [28]:
# Instruções para orientar o modelo a converter uma consulta em linguagem natural em código Python executável com a biblioteca Pandas
instruction_str = (
    "1. Converta a consulta para código Python executável usando Pandas.\n"
    "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
    "3. O código deve representar uma solução para a consulta.\n"
    "4. IMPRIMA APENAS A EXPRESSÃO.\n"
    "5. Não coloque a expressão entre aspas.\n")

# Prompt que será enviado ao modelo para que ela gere o código Pandas desejado
pandas_prompt_str = (
    "Você está trabalhando com um dataframe do pandas em Python chamado `df`.\n"
    "{colunas_detalhes}\n\n"
    "Este é o resultado de `print(df.head())`:\n"
    "{df_str}\n\n"
    "Siga estas instruções:\n"
    "{instruction_str}\n"
    "Consulta: {query_str}\n\n"
    "Expressão:"
)

# Prompt para guiar o modelo a sintetizar uma resposta com base nos resultados obtidos pela consulta Pandas
response_synthesis_prompt_str = (
   "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
   "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
   "Consulta: {query_str}\n\n"
   "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
   "Saída do Pandas: {pandas_output}\n\n"
   "Resposta:"
   "Ao final, exibir o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}"
)

In [29]:
# Função para obter uma descrição das colunas do DataFrame
def descricao_colunas(df):
  descricao = '\n'.join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
  return 'Aqui estão os detalhes das colunas do DataFrame:\n' + descricao

In [30]:
from llama_index.core import PromptTemplate
from llama_index.experimental.query_engine.pandas import PandasInstructionParser

In [ ]:
pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str,
    colunas_detalhes=descricao_colunas(df),
    df_str=df.head(5))

pandas_output_parser = PandasInstructionParser(df)

response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

llm = Groq(model='llama-3.3-70b-versatile', api_key=GROQ_API)

In [34]:
from llama_index.core.query_pipeline import (QueryPipeline as QP, Link, InputComponent)

ModuleNotFoundError: No module named 'llama_index.core.query_pipeline'